In [ ]:
!pip install -q groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 4.1 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata

In [ ]:
import os

from groq import Groq

client = Groq(
    api_key=userdata.get('GROQ_API_KEY'),
)

In [ ]:
! pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.2 MB/s eta 0:00:00


In [ ]:
! pip install pymupdf4llm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 74.8 MB/s eta 0:00:00


In [ ]:
from groq import Groq
import PyPDF2
import json
import time
import re
import pymupdf4llm
import os
import fitz

In [ ]:
os.path.exists("/content/drive/MyDrive/KnowledgeDatabase/R048r12e.pdf")

True

In [ ]:
from groq import Groq
import PyPDF2
import json
import time
import re

# Initialize Groq API client
client = Groq(
    api_key=userdata.get('GROQ_API_KEY'),
)

# Function to extract text from a PDF file page by page
def extract_text_from_pdf(pdf_path):
    text_pages = []
    with open(pdf_path, "rb") as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page in pdf_reader.pages:
            text = page.extract_text()
            if text:
                text_pages.append(text.strip())  # Remove extra spaces
    return text_pages

# Function to preprocess text (lowercase, remove special characters, etc.)
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespaces
    text = re.sub(r'[^a-z0-9\s\.,;:?!-]', '', text)  # Keep alphanumeric and punctuation
    return text

# Function to extract structured data using Groq API
def get_knowledge_graph(preprocessed_text, retries=3):
    for attempt in range(retries):
        try:
            response = client.chat.completions.create(
                messages=[
                    {
                        "role": "system",
                        "content": "You are an AI assistant that extracts structured knowledge from text. "
                                   "Your task is to extract entities and relationships from the given text, "
                                   "and return them in a structured JSON format as follows:\n\n"
                                   "{\n"
                                   '  "entities": [\n'
                                   '    {"name": "Entity1", "properties": {"property1": "value1", "property2": "value2"}},\n'
                                   '    {"name": "Entity2", "properties": {"propertyA": "valueA"}}\n'
                                   '  ],\n'
                                   '  "relationships": [\n'
                                   '    {"subject": "Entity1", "relationship": "relates_to", "object": "Entity2"}\n'
                                   '  ]\n'
                                   '}\n\n'
                                   "Ensure that each subject and object are individual entities."
                    },
                    {
                        "role": "user",
                        "content": f"""Extract entities and relationships from the following text:

                        {preprocessed_text}

                        Respond strictly with a valid JSON object and nothing else—no explanations, introductions, or extra text."""
                    }
                ],
                model="llama3-70b-8192",
                temperature=0.5,
                # max_tokens=512,  # Increased token limit
                top_p=1,
                stop=None,
                stream=False,
            )

            # Attempt to parse the response as JSON
            extracted_json = response.choices[0].message.content.strip()
            parsed_data = json.loads(extracted_json)

            # Ensure the output follows the expected structure
            if "entities" in parsed_data and "relationships" in parsed_data:
                return parsed_data

        except json.JSONDecodeError:
            print(f"Attempt {attempt+1}: Failed to parse JSON. Retrying...")
            time.sleep(2)  # Wait before retrying

        except Exception as e:
            print(f"Error during API call: {e}")
            return None

    print("Failed to extract structured data after retries.")
    return None

# Process PDF page by page
pdf_path = "/content/drive/MyDrive/KnowledgeDatabase/R048r12e.pdf"
text_pages = extract_text_from_pdf(pdf_path)

print("text-pages extraction is completed")

knowledge_graph = []

for idx, page_text in enumerate(text_pages):
    print(f"Processing Page {idx + 1}/{len(text_pages)}...")

    # Preprocess the text before sending to LLM
    preprocessed_text = preprocess_text(page_text)

    # Get knowledge graph data
    kg_data = get_knowledge_graph(preprocessed_text)

    if kg_data:
        knowledge_graph.append(kg_data)
        print(f"Extraction for Page {idx + 1} is completed.")
    else:
        print(f"Skipping Page {idx + 1} due to parsing issues.")

# Save the knowledge graph as a JSON file
output_file = "/content/drive/MyDrive/KnowledgeGraphResults/kgf.json"
with open(output_file, "w") as f:
    json.dump(knowledge_graph, f, indent=2)

print(f"Knowledge graph extraction completed. Results saved to {output_file}")


text-pages extraction is completed
Processing Page 1/131...
Extraction for Page 1 is completed.
Processing Page 2/131...
Failed to extract structured data after retries.
Skipping Page 2 due to parsing issues.
Processing Page 3/131...
Extraction for Page 3 is completed.
Processing Page 4/131...
Extraction for Page 4 is completed.
Processing Page 5/131...
Extraction for Page 5 is completed.
Processing Page 6/131...
Extraction for Page 6 is completed.
Processing Page 7/131...
Extraction for Page 7 is completed.
Processing Page 8/131...
Extraction for Page 8 is completed.
Processing Page 9/131...
Extraction for Page 9 is completed.
Processing Page 10/131...
Extraction for Page 10 is completed.
Processing Page 11/131...
Extraction for Page 11 is completed.
Processing Page 12/131...
Extraction for Page 12 is completed.
Processing Page 13/131...
Attempt 1: Failed to parse JSON. Retrying...
Extraction for Page 13 is completed.
Processing Page 14/131...
Extraction for Page 14 is completed.
Proc

# Using pymupdf4llm and Markdown for text extraction

In [ ]:


# Initialize Groq API client
client = Groq(
    api_key=userdata.get('GROQ_API_KEY'),
)

# Function to extract text from a PDF file page by page

def extract_text_from_pdf(pdf_path):
    text_pages = []

    # Open PDF once and get the number of pages
    doc = fitz.open(pdf_path)
    num_pages = len(doc)

    for page_num in range(num_pages):
        try:
            md_text = pymupdf4llm.to_markdown(pdf_path, pages=[page_num])
            if md_text:
                text_pages.append(md_text.strip())  # Remove extra spaces
        except Exception as e:
            print(f"Error processing page {page_num + 1}: {e}")  # Log errors but continue

    return text_pages


# Function to preprocess text (lowercase, remove special characters, etc.)

def preprocess_text(md_text):
    """
    Preprocesses Markdown text by performing the following steps:
    1. Cleans extra whitespace and newlines.
    2. Removes unwanted Markdown formatting (headers, bold, italic, inline code).
    3. Removes links and images.
    4. Removes document reference patterns like E/ECE/324/Rev.X/Add.Y/Rev.Z.

    :param md_text: str, Markdown text to be processed
    :return: str, Cleaned text
    """
    # Step 1: Remove extra whitespace and newlines
    md_text = re.sub(r'\n+', '\n', md_text.strip())
    md_text = re.sub(r'\s([?.!,:;"])(?=\s|$)', r'\1', md_text)

    # Step 2: Remove headers (lines starting with #)
    md_text = re.sub(r'^#+\s', '', md_text, flags=re.MULTILINE)

    # Step 3: Remove bold (**bold**) and italic (*italic*) markers
    md_text = re.sub(r'(\*\*|\*)', '', md_text)

    # Step 4: Remove inline code (e.g., `code`)
    md_text = re.sub(r'`(.*?)`', r'\1', md_text)

    # Step 5: Remove images (e.g., ![alt text](image_url))
    md_text = re.sub(r'!\[.*?\]\(.*?\)', '', md_text)

    # Step 6: Remove links (e.g., [text](url))
    md_text = re.sub(r'\[.*?\]\(.*?\)', '', md_text)

    # Step 7: Remove document reference patterns like E/ECE/324/Rev.X/Add.Y/Rev.Z
    md_text = re.sub(r'E/ECE/\S+', '', md_text)
    md_text = md_text.lower()

    return md_text


# Function to extract structured data using Groq API
def get_knowledge_graph(preprocessed_text, retries=3):
    for attempt in range(retries):
        try:
            response = client.chat.completions.create(
                messages=[
                    {
                        "role": "system",
                        "content": "You are an AI assistant that extracts structured knowledge from text. "
                                   "Your task is to extract entities and relationships from the given text, "
                                   "and return them in a structured JSON format as follows:\n\n"
                                   "{\n"
                                   '  "entities": [\n'
                                   '    {"name": "Entity1", "properties": {"property1": "value1", "property2": "value2"}},\n'
                                   '    {"name": "Entity2", "properties": {"propertyA": "valueA"}}\n'
                                   '  ],\n'
                                   '  "relationships": [\n'
                                   '    {"subject": "Entity1", "relationship": "relates_to", "object": "Entity2"}\n'
                                   '  ]\n'
                                   '}\n\n'
                                   "Ensure that each subject and object are individual entities."
                    },
                    {
                        "role": "user",
                        "content": f"""Extract entities and relationships from the following text:

                        {preprocessed_text}

                        Respond strictly with a valid JSON object and nothing else—no explanations, introductions, or extra text."""
                    }
                ],
                model="llama3-70b-8192",
                temperature=0.5,
                # max_tokens=512,  # Increased token limit
                top_p=1,
                stop=None,
                stream=False,
            )

            # Attempt to parse the response as JSON
            extracted_json = response.choices[0].message.content.strip()
            parsed_data = json.loads(extracted_json)

            # Ensure the output follows the expected structure
            if "entities" in parsed_data and "relationships" in parsed_data:
                return parsed_data

        except json.JSONDecodeError:
            print(f"Attempt {attempt+1}: Failed to parse JSON. Retrying...")
            time.sleep(2)  # Wait before retrying

        except Exception as e:
            print(f"Error during API call: {e}")
            return None

    print("Failed to extract structured data after retries.")
    return None

# Process PDF page by page
pdf_path = "/content/drive/MyDrive/KnowledgeDatabase/R048r12e.pdf"
text_pages = extract_text_from_pdf(pdf_path)

print("text-pages extraction is completed")

knowledge_graph = []

for idx, page_text in enumerate(text_pages):
    print(f"Processing Page {idx + 1}/{len(text_pages)}...")

    # Preprocess the text before sending to LLM
    preprocessed_text = preprocess_text(page_text)

    # Get knowledge graph data
    kg_data = get_knowledge_graph(preprocessed_text)

    if kg_data:
        knowledge_graph.append(kg_data)
        print(f"Extraction for Page {idx + 1} is completed.")
    else:
        print(f"Skipping Page {idx + 1} due to parsing issues.")

# Save the knowledge graph as a JSON file
output_file = "/content/drive/MyDrive/KnowledgeGraphResults/kgf2.json"
with open(output_file, "w") as f:
    json.dump(knowledge_graph, f, indent=2)

print(f"Knowledge graph extraction completed. Results saved to {output_file}")


Processing /content/drive/MyDrive/KnowledgeDatabase/R048r12e.pdf...
[                                        ] (0/1)========================================[========================================] (1/1)]
Processing /content/drive/MyDrive/KnowledgeDatabase/R048r12e.pdf...
[                                        ] (0/1)========================================[========================================] (1/1)]
Processing /content/drive/MyDrive/KnowledgeDatabase/R048r12e.pdf...
[                                        ] (0/1)========================================[========================================] (1/1)]
Processing /content/drive/MyDrive/KnowledgeDatabase/R048r12e.pdf...
[          